In [8]:
import pandas as pd
import requests
import json
import os

In [9]:
genes = ['GARS1', 'HARS1', 'CILK1', 'BPNT2', 'KARS1', 'DNAAF11', 'MARS1', 'QARS1', 'SEPTIN12', 'TAFAZZIN', 'STING1', 'AARS1', 'ODAD2', 'MTRFR', 'ODAD1']

In [10]:
query = '''
{
  gene(gene_symbol: "%s", reference_genome: GRCh38) {
    gene_id
    start
    stop
    chrom
    variants(dataset: gnomad_r3) {
      variant_id
      pos
      ref
      alt
      reference_genome
      transcript_consequence {
        major_consequence
        polyphen_prediction
      }
    }
  }
}
'''

In [11]:
end_point = "https://gnomad.broadinstitute.org/api/"

In [15]:
import time

def getdata(gene):
    """
    Returns the response from gnomad API for a gene as well as the attempt number. If 429 errors, then retries else 
    returns -1, {}.
    If all retries fail, returns, -2, {}.
    """
    max_retries = 10
    initial_delay = 1
    backoff_factor = 2
    
    
    
    for attempt in range(max_retries):
        response = requests.post(end_point, data={'query': query % (gene)}, timeout=None)
        print(f'attempt: {attempt + 1}')
        if response.status_code == 200:
            if 'errors' not in response.json():
                return response.json()['data']
            else:
                return {'error': response.json()['errors']}
        elif response.status_code == 429:
            delay = initial_delay * (backoff_factor ** (attempt + 1))
            if attempt < max_retries - 1:
                time.sleep(delay)   
        else:
            return {'error': response.status_code}
    return {'error': -2}

In [13]:
gnomad_data = []
from IPython.display import clear_output
while genes:
    gene = genes.pop(0)
    print(len(genes))
    print(gene)
    gene_data = getdata(gene)
    if 'error' in gene_data:
        if not isinstance(gene_data['error'], list):
            genes.append(gene)
    else:
        gnomad_data.append(gene_data)
    clear_output()
    time.sleep(0.1)

In [14]:
import json

jsonpath = './output/gnomad_data_more.json'

with open(jsonpath, 'w') as file:
    json.dump(gnomad_data, file)